In [ ]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable
import re

def forLoop(code):
    fc = []
    idx = None
    temp_counter = 1

    def new_temp():
        nonlocal temp_counter
        temp = f"t{temp_counter}"
        temp_counter += 1
        return temp

    for i in range(len(code)):
        line = code[i]

        if 'for' in line:
            idx = 0
            start_idx = line.index('(')
            end_idx = line.index(')')
            parts = line[start_idx+1:end_idx].split(';')

            initialization = parts[0].strip()
            condition = parts[1].strip()
            updation = parts[2].strip()

            if '=' in initialization:
                parts = initialization.split('=')
                var_name = parts[0].strip()
                operation = parts[1].strip()

                if '+' in operation or '-' in operation or '*' in operation or '/' in operation:
                    operands = operation.split()
                    result_temp = new_temp()
                    fc.append(f"{result_temp} = {operands[0]} {operands[1]} {operands[2]}")
                    for j in range(3, len(operands), 2):
                        temp_var = new_temp()
                        fc.append(f"{temp_var} = {result_temp} {operands[j]} {operands[j+1]}")
                        result_temp = temp_var
                    fc.append(f"{var_name} = {result_temp}")
                else:
                    fc.append(f"{var_name} = {operation}")
                idx = idx+1 # for m1.quad


            if "<=" in condition:
                conditions = condition.split('<=')
                fc.append(f"if ({conditions[0].strip()}) <= {conditions[1]} goto {idx+5}")
                idx = idx + 1
                fc.append(f"goto {idx+10}")
                idx = idx+1 # for m2.quad


            if '=' in updation:
                parts = updation.split('=')
                var_name = parts[0].strip()
                operation = parts[1].strip()
                operator = re.findall(r'[+\-*\/]', operation)
                if operator:
                    operator = operator[0]

                    operands = re.split(r'\s*[-+\/*]\s*', operation)
                    if len(operands) >= 2:
                        result_temp = new_temp()
                        fc.append(f"{result_temp} = {operands[0]} {operator} {operands[1]}")
                        for j in range(2, len(operands)):
                            temp_var = new_temp()
                            fc.append(f"{temp_var} = {result_temp} {operator} {operands[j]}")
                            result_temp = temp_var
                        fc.append(f"{var_name} = {result_temp}")
                    else:
                        fc.append(f"{var_name} = {operation}")
                else:
                    fc.append(f"{var_name} = {operation}")
                idx = idx + 1
                idx = idx + 1 # for m3.quad
                fc.append(f"goto {idx-4}")

        else:
            if '=' in line:
                parts = line.split('=')
                var_name = parts[0].strip()
                operation = parts[1].strip()

                operator = re.findall(r'[+\-*\/]', operation)
                if operator:
                    operator = operator[0]

                    operands = re.split(r'\s*[-+\/*]\s*', operation)
                    if len(operands) >= 2:
                        result_temp = new_temp()
                        fc.append(f"{result_temp} = {operands[0]} {operator} {operands[1]}")
                        for j in range(2, len(operands)):
                            temp_var = new_temp()
                            fc.append(f"{temp_var} = {result_temp} {operator} {operands[j]}")
                            result_temp = temp_var
                        fc.append(f"{var_name} = {result_temp}")
                    else:
                        fc.append(f"{var_name} = {operation}")
                else:
                    fc.append(f"{var_name} = {operation}")

            else:
                fc.append(line)

    fc.append(f"goto {idx-2}")

    fc.append(f"END")

    return fc

with open('forloop.txt') as f:
    code = f.readlines()

ans = []
for i in range(len(code)):
    if code[i] != '\n':
        if code[i][-1] == '\n':
            ans.append(code[i][:-1].strip())
        else:
            ans.append(code[i].strip())

fans = forLoop(ans)

print('\nTAC for FOR loop is:')
show = PrettyTable()
show.field_names = ['Index','TAC']
for i in range(len(fans)):
    show.add_row([i,fans[i]])

print(show)


TAC for FOR loop is:
+-------+---------------------+
| Index |         TAC         |
+-------+---------------------+
|   0   |        i = 1        |
|   1   | if (i) <= 20 goto 6 |
|   2   |       goto 12       |
|   3   |      t1 = i + 1     |
|   4   |        i = t1       |
|   5   |        goto 1       |
|   6   |      t2 = y + z     |
|   7   |        x = t2       |
|   8   |        b = c        |
|   9   |      t3 = i + 1     |
|   10  |        j = t3       |
|   11  |        goto 3       |
|   12  |         END         |
+-------+---------------------+
